In [19]:
import json
import numpy as np

def read_labels():

  path = "Labeled Data\\Labels.txt"
  data = []
  labels = {}

  with open(path, "r") as f:
    data = f.readlines()

  for line in data:
    temp = line.split()
    sublist = temp[2:11]
    temp[10] = temp[10].replace("\n", "")
    score = sum(float(i) for i in sublist)
    labels[temp[0]] = score

  return labels

def class_assesment(scores, setting):

  labels = []

  if setting == 4:
    class_instances = [0, 0, 0, 0]
    for score in scores:
      if score >= 75:
        labels.append([0])
        class_instances[0] += 1
      elif score >= 50:
        labels.append([1])
        class_instances[1] += 1
      elif score >= 25:
        labels.append([2])
        class_instances[2] += 1
      else:
        labels.append([3])
        class_instances[3] += 1
  elif setting == 6:
    class_instances = [0, 0, 0, 0, 0, 0]
    for score in scores:
      if score >= 83.5:
        labels.append([0])
        class_instances[0] += 1
      elif score >= 67:
        labels.append([1])
        class_instances[1] += 1
      elif score >= 50.5:
        labels.append([2])
        class_instances[2] += 1
      elif score >= 34:
        labels.append([3])
        class_instances[3] += 1
      elif score >= 17.5:
        labels.append([4])
        class_instances[4] += 1
      else:
        labels.append([5])
        class_instances[5] += 1
  elif setting == 8:
    class_instances = [0, 0, 0, 0, 0, 0, 0, 0]
    for score in scores:
      if score >= 87.5:
        labels.append([0])
        class_instances[0] += 1
      elif score >= 75:
        labels.append([1])
        class_instances[1] += 1
      elif score >= 62.5:
        labels.append([2])
        class_instances[2] += 1
      elif score >= 50:
        labels.append([3])
        class_instances[3] += 1
      elif score >= 37.5:
        labels.append([4])
        class_instances[4] += 1
      elif score >= 25:
        labels.append([5])
        class_instances[5] += 1
      elif score >= 12.5:
        labels.append([6])
        class_instances[6] += 1
      else:
        labels.append([7])
        class_instances[7] += 1
  else:
    class_instances = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for score in scores:
      if score >= 90:
        labels.append([0])
        class_instances[0] += 1
      elif score >= 80:
        labels.append([1])
        class_instances[1] += 1
      elif score >= 70:
        labels.append([2])
        class_instances[2] += 1
      elif score >= 60:
        labels.append([3])
        class_instances[3] += 1
      elif score >= 50:
        labels.append([4])
        class_instances[4] += 1
      elif score >= 40:
        labels.append([5])
        class_instances[5] += 1
      elif score >= 30:
        labels.append([6])
        class_instances[6] += 1
      elif score >= 20:
        labels.append([7])
        class_instances[7] += 1
      elif score >= 10:
        labels.append([8])
        class_instances[8] += 1
      else:
        labels.append([9])
        class_instances[9] += 1


  return labels, class_instances


def read_profile_embedding():
  with open("Non textual features\\profile_tweet_non_textual_embedding.json", 'r') as f:
    temp = json.load(f)
    # temp = ast.literal_eval(temp)
    embed = {x: v for x, v in temp.items()}
    f.close()
  return embed

def read_mention_embedding():
  with open("Textual features\\Mentions\\mentions.json", 'r') as f:
    temp = json.load(f)
    # temp = ast.literal_eval(temp)
    embed = {x: v for x, v in temp.items()}
    f.close()
  return embed

def read_tweet_text_embedding():
  with open("Textual features\\Tweets\\tweets_embeddings_compressed.json", 'r') as f:
    temp = json.load(f)
    # temp = ast.literal_eval(temp)
    embed = {x: v for x, v in temp.items()}
    f.close()
  return embed

In [20]:
import functools
import operator

X = []
Y = []

embbedings = []

labels = read_labels()

prof_embed = read_profile_embedding()
mention_embed = read_mention_embedding()
tweet_embed = read_tweet_text_embedding()
accounts = mention_embed.keys()

a = set(mention_embed.keys()) - set(prof_embed.keys())

for acc in accounts:
  if acc != "heavy.com":

    dic = {}
    vec = []

    vec_2 = tweet_embed[acc]

    vec_3 = functools.reduce(operator.concat, mention_embed[acc])
    if acc == "meanshealth.com":
      acc = "menshealth.com"

    vec_1 = prof_embed[acc]
    score = labels[acc]
    vec = vec_1 + vec_3 + vec_2
    # vec = vec_1
    # vec = vec_1 + vec_3

    dic["name"] = acc
    dic["embedding"] = vec
    dic["score"] = score
    embbedings.append(dic)

for datapoint in embbedings:
  X.append(datapoint["embedding"])
  Y.append(datapoint["score"])



In [3]:
num_classes = 10
Y_class, instances = class_assesment(Y, num_classes)

In [22]:
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from imblearn.over_sampling import SMOTE

min_max_scaler = MinMaxScaler()
X_ = min_max_scaler.fit_transform(X)

In [ ]:
if num_classes > 6:
    smote = SMOTE(k_neighbors=3)
else:
    smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(X_, Y_class)
print(f'Original dataset shape {len(Y_class)}')
print(f'Resample dataset shape {len(y_smote)}')

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(np.array(x_smote), np.array(y_smote), test_size=0.2, random_state=1)


In [24]:
def get_model():
  hidden_layer_1 = 256
  hidden_layer_2 = 256
  hidden_layer_3 = 64

  model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(51,), name="input_layer"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=hidden_layer_1, kernel_initializer='he_normal', activation='relu', name="hidden_layer_1"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=hidden_layer_2, kernel_initializer='he_normal', activation='relu', name="hidden_layer_2"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units=hidden_layer_3, activation='relu', name="hidden_layer_3"),
    keras.layers.Dense(num_classes, activation='softmax')
  ])

  model.summary()

  return model

In [ ]:
model = get_model()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

opt_1 = keras.optimizers.Adagrad(learning_rate=0.1)
opt_2 = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=opt_2, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, verbose=3, restore_best_weights=True, mode='min')

model.fit(X_train,
          y_train,
          epochs=2000,
          batch_size = 16,
          validation_split=0.2,
          shuffle=True,
          callbacks=callback
)

In [ ]:
from sklearn.metrics import classification_report

y_test_pred = np.argmax(model.predict(X_test), axis=1)

print(classification_report(y_test, y_test_pred, digits=4))

In [ ]:
num_itration = 10

for i in range(num_itration):

  model = get_model()
  lr_schedule = keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=1e-2,
      decay_steps=10000,
      decay_rate=0.9)

  opt_1 = keras.optimizers.Adagrad(learning_rate=0.1)
  opt_2 = keras.optimizers.Adam(learning_rate=lr_schedule)

  model.compile(optimizer=opt_2, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
  callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, verbose=3, restore_best_weights=True, mode='min')

  model.fit(X_train,
            y_train,
            epochs=2000,
            batch_size = 16,
            validation_split=0.2,
            shuffle=True,
            callbacks=callback
  )

  y_test_pred = np.argmax(model.predict(X_test), axis=1)
  
  log = classification_report(y_test, y_test_pred, digits=4)

  path = 'Classifier Results\\' + str(num_classes) + '_class.txt'

  with open(path, "a") as f:
    f.write(log)
    f.write('\n')
  f.close()